# 2023-12-05

In [1]:
from aocd.models import Puzzle


## Initial solution
Woke up att 5:50am (~Midnight UTC-5) and did it on time. 
```
      --------Part 1--------   --------Part 2--------
Day       Time   Rank  Score       Time   Rank  Score
  5   01:20:56  10089      0   04:25:14   9235      0
```

### Puzzle 1
Sheesh, this was embarassing. The idea her is that we first create a mapping table and then we traverse each seed and follow it through recursively until it reaches the "location" type. The lookup operations are instant and work fast.

Not sure why I was so slow. 

In [136]:
from collections import defaultdict

puzzle = Puzzle(2023, 5)

t_table = defaultdict(dict)
chunks = [x.splitlines() for x in puzzle.input_data.split("\n\n")]
seeds, mappings = chunks[0][0].split(": ")[1:][0].split(" "), chunks[1:]

for mapping in mappings:
    source_key, target_key = mapping[0].split(" map")[0].split("-to-")
    t_table[source_key].update({"ranges": []})
    t_table[source_key].update({"target": target_key})
    for ranges in mapping[1:]:
        dest_start, source_start, length = map(lambda x: int(x), ranges.split(" "))
        offset = dest_start - source_start
        t_table[source_key]["ranges"].append({range(source_start, source_start + length): offset})

def traverse(init_value, init_type="seed", target_type="location"):
    if init_type == target_type:
        return init_value
    ranges = t_table.get(init_type).get("ranges")
    target = t_table.get(init_type).get("target")
    for r in ranges:
        if init_value in list(r.keys())[0]:
            offset = list(r.values())[0]
            #print(f"{init_type} {init_value} -> {target} {init_value + offset}, offset: {offset} using {r}")
            return traverse(init_value + offset, init_type=target)
    #print(f"{init_type} {init_value} -> {target} {init_value}")
    return traverse(init_value, init_type=target)

seeds = [int(x) for x in seeds]
puzzle.answer_a = min([traverse(seed) for seed in seeds])


### Part 2
Here I got really stuck, I didnt understand how to think of the problem until much later. At first I attempted to check if there were sequences that were repeated and thought one could creaet a shortcut mapping, think memoization. But a check revealed that that was not what was causing my solution to be slow.

It was only later than I realised that one could interpret is as splicing ranges and just applying the same logic on a range.

This solution does the same as the above pretty much, but we first calculate any potential overlap between the ranges, if there is an overlap, we apply the offset to the range and transform it to its new range with its new type. If there was remains of the original range we then call the function recursively with the same parameters to check if there are any more overlaps with other ranges for the type.

#### ChatGPT Summary
**Range Splitting and Transformation:** It splits an initial range into overlapping and non-overlapping segments relative to a set of predefined ranges. For each overlapping part, it applies an offset to transform the range into the target type.

**Recursive Processing:** The function handles the non-overlapping parts of the initial range recursively, ensuring that the entire range is transformed according to the mapping rules.

**Aggregation and Optimization:** After processing, the function aggregates all the transformed ranges and selects the one with the smallest starting point. This ensures the most optimal transformation result.

In [137]:
def split_initial_range(range1, range2):
    start_overlap = max(range1.start, range2.start)
    end_overlap = min(range1.stop, range2.stop)
    overlap = range(start_overlap, end_overlap) if start_overlap < end_overlap else None

    remaining_start = range(range1.start, start_overlap) if range1.start < start_overlap else None
    remaining_end = range(end_overlap, range1.stop) if end_overlap < range1.stop else None

    return overlap, remaining_start, remaining_end

def traverse(init_range, init_type="seed", target_type="location"):
    if init_type == target_type:
        return init_range
    ranges = t_table.get(init_type).get("ranges")
    target = t_table.get(init_type).get("target")
    possible_minimums = []
    for r in ranges:
        test_range = list(r.keys())[0]
        overlap, remaining_start, remaining_end = split_initial_range(init_range, test_range)
        if overlap:
            offset = list(r.values())[0]
            possible_minimums.append(traverse(range(overlap.start + offset, overlap.stop + offset), init_type=target))
            for non_overlap in [remaining_start, remaining_end]:
                if non_overlap:
                    possible_minimums.append(traverse(non_overlap, init_type=init_type))
            break
    if len(possible_minimums) == 0:
        possible_minimums.append(traverse(init_range, init_type=target))
    return min(possible_minimums, key=lambda r: r.start)

seeds = [int(x) for x in seeds]
puzzle.answer_b = min([traverse(r).start for r in [range(x, x + y) for x,y in zip(seeds[::2], seeds[1::2])]])


### Part 2 with debugging

In [ ]:

def traverse(init_range, init_type="seed", target_type="location"):
    if init_type == target_type:
        return init_range
    ranges = t_table.get(init_type).get("ranges")
    target = t_table.get(init_type).get("target")
    possible_minimums = []
    for r in ranges:
        test_range = list(r.keys())[0]
        overlap, remaining_start, remaining_end = split_initial_range(init_range, test_range)
        #print(f"Trying {init_type} {init_range} on {test_range}: Overlap {overlap}, remaining_start {remaining_start}, remaining_end {remaining_end}")
        if overlap:
            #print("There is an overlap")
            offset = list(r.values())[0]
            #print(f"Change: {init_type} {overlap} -> {target} {range(overlap.start + offset, overlap.stop + offset)}, offset: {offset} using {test_range}")
            possible_minimums.append(traverse(range(overlap.start + offset, overlap.stop + offset), init_type=target))
            #print("Trying remaining start and end")
            for non_overlap in [remaining_start, remaining_end]:
                if non_overlap:
                    #print("There are non-overlap remaining")
                    #print(f"Re-trying on {init_type} {init_range} with remains: {init_type} {non_overlap}")
                    possible_minimums.append(traverse(non_overlap, init_type=init_type))
            break
    if len(possible_minimums) == 0:
        #print(f"Changing: {init_type} {init_range} -> {target} {init_range}")
        possible_minimums.append(traverse(init_range, init_type=target))
    #print(init_range, init_type, possible_minimums)
    #print("possible minimums", possible_minimums, min(possible_minimums, key=lambda r: r.start))
    return min(possible_minimums, key=lambda r: r.start)

seeds = [int(x) for x in seeds]
puzzle.answer_b = min([traverse(r).start for r in [range(x, x + y) for x,y in zip(seeds[::2], seeds[1::2])]])
